# 决策树 概述
决策树（Decision Tree）算法主要用来处理分类问题，是最经常使用的数据挖掘算法之一。

# 决策树 原理
构造决策树的方法：
createBranch递归调用
```
createBranch()
    检测数据集中的所有数据的分类标签是否相同:
        If so return 类标签
        Else:
            寻找划分数据集的最好特征（划分之后信息熵最小，也就是信息增益最大的特征）
            划分数据集
            创建分支节点
                for 每个划分的子集
                    调用函数 createBranch （创建分支的函数）并增加返回结果到分支节点中
            return 分支节点
```

# 决策树 开发流程
1. 收集数据：可以使用任何方法。
2. 准备数据：树构造算法只适用于标称型数据，因此数值型数据必须离散化。
3. 分析数据：可以使用任何方法，构造树完成之后，我们应该检查图形是否符合预期。
4. 训练算法：构造树的数据结构。
5. 测试算法：使用经验树计算错误率。（经验树没有搜索到较好的资料，有兴趣的同学可以来补充）
6. 使用算法：此步骤可以适用于任何监督学习算法，而使用决策树可以更好地理解数据的内在含义。

# 决策树 特点
- 优点：计算复杂度不高，输出结果易于理解，对中间值的缺失不敏感，可以处理不相关特征数据。
- 缺点：可能会产生过度匹配问题。
- 适用数据类型：数值型和标称型。

# 示例一 鱼的分类

## 所有类别的香农信息熵
$$H = - \sum\nolimits_{i=1}^nP(x_{i})\log_{2}P(x_{i})$$

In [27]:
# 程序清单3-1 计算给定数据的香农熵
from math import log
def calcShannonEnt(dataset):
    # 求样本数
    numEntries = len(dataset)
    #计算标签label出现的此数
    labelCounts = {}
    for featVec in dataset:
        #读训练集标签
        currentLabel = featVec[-1]
        #如果键值不存在，则创建
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]/numEntries)
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt


In [28]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [29]:
myDat , labels = createDataSet()
myDat
calcShannonEnt(myDat)
myDat[0][-1] = 'maybe'
myDat
calcShannonEnt(myDat)

1.3709505944546687

In [37]:
def splitDataSet(dataSet, axis, value):
    """
    Desc:
        把axis列值为value的数据集
    Args:
        dataSet 数据集
        axis 列的索引
        value 待分类的特征
    return：
        axis 列为value的数据集
    """
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet       
        

In [45]:
# test splitDataSet
myDat, labels = createDataSet()
print(myDat)
print(labels)
#第0列为1的数据
result1 = splitDataSet(myDat, 0, 1)
print(result1)
#第0列为0的数据
result2 = splitDataSet(myDat, 0, 0)
print(result2)
len(myDat[0])
[example[0] for example in myDat]

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['no surfacing', 'flippers']
[[1, 'yes'], [1, 'yes'], [0, 'no']]
[[1, 'no'], [1, 'no']]


[1, 1, 1, 0, 0]

In [49]:
def chooseBestFeatureToSplit(dataSet):
    """
    Desc:
        选择最好的特征
    Args:
        dataSet 数据集
    returns：
        bestFeature 最优的特之列
    """
    # 第一行有多少列的 Feature, 最后一列是label
    numFeatures = len(dataSet[0]) - 1
    # 数据集的原始信息熵
    baseEntropy = calcShannonEnt(dataSet)
    # 最优的信息增益值, 和最优的Featurn编号
    bestInfoGain, bestFeature = 0.0, -1
    for i in range(numFeatures):
        # 获取对应的feature下的所有数据
        featList = [example[i] for example in dataSet]
        #元素去重
        uniqueVals = set(featList)
        #创建临时信息熵
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            #计算第i列，值为value的权重
            prob = len(subDataSet)/float(len(dataSet))
            ###########信息熵乘以权重，目的是样本数多的特征占比大#############
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        print ('infoGain=', infoGain, 'bestFeature=', i, baseEntropy, newEntropy)
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature
        

In [50]:
chooseBestFeatureToSplit(myDat)

infoGain= 0.4199730940219749 bestFeature= 0 0.9709505944546686 0.5509775004326937
infoGain= 0.17095059445466854 bestFeature= 1 0.9709505944546686 0.8


0